<a href="https://colab.research.google.com/github/TheKoray18/Ridge-Lasso-Elastic-Net-Regression/blob/main/Ridge_Lasso_ElasticNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive

drive.mount('/content/drive/')

Mounted at /content/drive/


In [3]:
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings('ignore')

In [34]:
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/column_2C_weka.csv')

data = df.copy()

In [ ]:
data.head()

,pelvic_incidence,pelvic_tilt numeric,lumbar_lordosis_angle,sacral_slope,pelvic_radius,degree_spondylolisthesis,class
0,63.027818,22.552586,39.609117,40.475232,98.672917,-0.254400,Abnormal
1,39.056951,10.060991,25.015378,28.995960,114.405425,4.564259,Abnormal
2,68.832021,22.218482,50.092194,46.613539,105.985135,-3.530317,Abnormal
3,69.297008,24.652878,44.311238,44.644130,101.868495,11.211523,Abnormal
4,49.712859,9.652075,28.317406,40.060784,108.168725,7.918501,Abnormal


In [ ]:
data.tail()

,pelvic_incidence,pelvic_tilt numeric,lumbar_lordosis_angle,sacral_slope,pelvic_radius,degree_spondylolisthesis,class
305,47.903565,13.616688,36.000000,34.286877,117.449062,-4.245395,Normal
306,53.936748,20.721496,29.220534,33.215251,114.365845,-0.421010,Normal
307,61.446597,22.694968,46.170347,38.751628,125.670725,-2.707880,Normal
308,45.252792,8.693157,41.583126,36.559635,118.545842,0.214750,Normal
309,33.841641,5.073991,36.641233,28.767649,123.945244,-0.199249,Normal


In [ ]:
data.isnull().any()

pelvic_incidence            False
pelvic_tilt numeric         False
lumbar_lordosis_angle       False
sacral_slope                False
pelvic_radius               False
degree_spondylolisthesis    False
class                       False
dtype: bool

In [ ]:
data.isnull().any().sum()

0

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 310 entries, 0 to 309
Data columns (total 7 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   pelvic_incidence          310 non-null    float64
 1   pelvic_tilt numeric       310 non-null    float64
 2   lumbar_lordosis_angle     310 non-null    float64
 3   sacral_slope              310 non-null    float64
 4   pelvic_radius             310 non-null    float64
 5   degree_spondylolisthesis  310 non-null    float64
 6   class                     310 non-null    object 
dtypes: float64(6), object(1)
memory usage: 17.1+ KB


In [ ]:
data.describe().T

,count,mean,std,min,25%,50%,75%,max
pelvic_incidence,310.0,60.496653,17.236520,26.147921,46.430294,58.691038,72.877696,129.834041
pelvic_tilt numeric,310.0,17.542822,10.008330,-6.554948,10.667069,16.357689,22.120395,49.431864
lumbar_lordosis_angle,310.0,51.930930,18.554064,14.000000,37.000000,49.562398,63.000000,125.742385
sacral_slope,310.0,42.953831,13.423102,13.366931,33.347122,42.404912,52.695888,121.429566
pelvic_radius,310.0,117.920655,13.317377,70.082575,110.709196,118.268178,125.467674,163.071041
degree_spondylolisthesis,310.0,26.296694,37.559027,-11.058179,1.603727,11.767934,41.287352,418.543082


In [ ]:
data.corr()

,pelvic_incidence,pelvic_tilt numeric,lumbar_lordosis_angle,sacral_slope,pelvic_radius,degree_spondylolisthesis
pelvic_incidence,1.000000,0.629199,0.717282,0.814960,-0.247467,0.638743
pelvic_tilt numeric,0.629199,1.000000,0.432764,0.062345,0.032668,0.397862
lumbar_lordosis_angle,0.717282,0.432764,1.000000,0.598387,-0.080344,0.533667
sacral_slope,0.814960,0.062345,0.598387,1.000000,-0.342128,0.523557
pelvic_radius,-0.247467,0.032668,-0.080344,-0.342128,1.000000,-0.026065
degree_spondylolisthesis,0.638743,0.397862,0.533667,0.523557,-0.026065,1.000000


In [ ]:
columns_name = data.columns
columns_name

Index(['pelvic_incidence', 'pelvic_tilt numeric', 'lumbar_lordosis_angle',
       'sacral_slope', 'pelvic_radius', 'degree_spondylolisthesis', 'class'],
      dtype='object')

# Data Preprocessing

In [36]:
data['class'] = pd.Categorical(data['class'])

In [38]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 310 entries, 0 to 309
Data columns (total 7 columns):
 #   Column                    Non-Null Count  Dtype   
---  ------                    --------------  -----   
 0   pelvic_incidence          310 non-null    float64 
 1   pelvic_tilt numeric       310 non-null    float64 
 2   lumbar_lordosis_angle     310 non-null    float64 
 3   sacral_slope              310 non-null    float64 
 4   pelvic_radius             310 non-null    float64 
 5   degree_spondylolisthesis  310 non-null    float64 
 6   class                     310 non-null    category
dtypes: category(1), float64(6)
memory usage: 15.1 KB


- Abnormal = 1
- Normal = 0

In [67]:
data['class'] = [1 if i == 'Abnormal' else 0 for i in data['class']]

data['class']

0      1
1      1
2      1
3      1
4      1
      ..
305    0
306    0
307    0
308    0
309    0
Name: class, Length: 310, dtype: int64

In [68]:
data.head()

,pelvic_incidence,pelvic_tilt numeric,lumbar_lordosis_angle,sacral_slope,pelvic_radius,degree_spondylolisthesis,class
0,63.027818,22.552586,39.609117,40.475232,98.672917,-0.254400,1
1,39.056951,10.060991,25.015378,28.995960,114.405425,4.564259,1
2,68.832021,22.218482,50.092194,46.613539,105.985135,-3.530317,1
3,69.297008,24.652878,44.311238,44.644130,101.868495,11.211523,1
4,49.712859,9.652075,28.317406,40.060784,108.168725,7.918501,1


In [69]:
y = data['pelvic_incidence']

y.head()

0    63.027818
1    39.056951
2    68.832021
3    69.297008
4    49.712859
Name: pelvic_incidence, dtype: float64

In [70]:
x = data.drop('pelvic_incidence',axis=1)

x.head()

,pelvic_tilt numeric,lumbar_lordosis_angle,sacral_slope,pelvic_radius,degree_spondylolisthesis,class
0,22.552586,39.609117,40.475232,98.672917,-0.254400,1
1,10.060991,25.015378,28.995960,114.405425,4.564259,1
2,22.218482,50.092194,46.613539,105.985135,-3.530317,1
3,24.652878,44.311238,44.644130,101.868495,11.211523,1
4,9.652075,28.317406,40.060784,108.168725,7.918501,1


In [71]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.33,random_state=18)

In [72]:
print("x_train :" + str(x_train.shape))
print("x_test :" + str(x_test.shape))
print("y_train :" + str(y_train.shape))
print("y_test :" + str(y_test.shape))

x_train :(207, 6)
x_test :(103, 6)
y_train :(207,)
y_test :(103,)


# Ridge Regresyon 

In [73]:
from sklearn.linear_model import Ridge

ridge_model = Ridge(alpha=0.1)

In [74]:
ridge_model.fit(x_train,y_train)

Ridge(alpha=0.1, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001)

In [75]:
y_pred = ridge_model.predict(x_test)

In [81]:
y_pred[:10]

array([61.4464988 , 67.5131917 , 52.2048688 , 53.43289168, 89.83457142,
       71.00200349, 37.90395374, 55.08088646, 63.79242101, 54.1248425 ])

In [84]:
y_test.head()

307    61.446597
201    67.513053
134    52.204693
6      53.432928
224    89.834676
Name: pelvic_incidence, dtype: float64

In [85]:
y_test = y_test.reset_index(drop=True)

y_test.head()

0    61.446597
1    67.513053
2    52.204693
3    53.432928
4    89.834676
Name: pelvic_incidence, dtype: float64

In [89]:
for i in range(10):

  print(f"Gerçek Değer : {y_test[i]} - Model Tahmini : {y_pred[i]}")

Gerçek Değer : 61.44659663 - Model Tahmini : 61.44649880364714
Gerçek Değer : 67.51305267 - Model Tahmini : 67.51319170382237
Gerçek Değer : 52.20469309 - Model Tahmini : 52.20486880012193
Gerçek Değer : 53.43292815 - Model Tahmini : 53.43289167673599
Gerçek Değer : 89.83467631 - Model Tahmini : 89.83457142024761
Gerçek Değer : 71.00194076 - Model Tahmini : 71.00200348533482
Gerçek Değer : 37.90391014 - Model Tahmini : 37.90395374195622
Gerçek Değer : 55.08076562 - Model Tahmini : 55.08088646364111
Gerçek Değer : 63.79242525 - Model Tahmini : 63.792421007463815
Gerçek Değer : 54.12492019 - Model Tahmini : 54.12484249540152


In [87]:
ridge_model.score(x_test,y_test)

0.9999999999720163

- Mean Squared Error 

In [88]:
from sklearn.metrics import mean_squared_error

mse_score = mean_squared_error(y_test,y_pred)

print(f"Mean Squared Error: {mse_score}")

Mean Squared Error: 7.604591498733154e-09


In [119]:
np.sqrt(mse_score)

8.720430894590677e-05

## Model Tuning - Cross Validation 

- Model Tuning işlemi yaparken lambda parametresini optimize edeceğiz.
- O yüzden model için en iyi değeri bulması için lambda değerleri oluşturuyoruz.

In [92]:
lamb_values = 10 ** np.linspace(10,-1,50) * 0.5

lamb_values

array([5.00000000e+09, 2.98181166e+09, 1.77824015e+09, 1.06047544e+09,
       6.32427608e+08, 3.77156003e+08, 2.24921633e+08, 1.34134790e+08,
       7.99929360e+07, 4.77047738e+07, 2.84493301e+07, 1.69661089e+07,
       1.01179482e+07, 6.03396320e+06, 3.59842837e+06, 2.14596713e+06,
       1.27977396e+06, 7.63208984e+05, 4.55149089e+05, 2.71433772e+05,
       1.61872877e+05, 9.65348864e+04, 5.75697700e+04, 3.43324423e+04,
       2.04745753e+04, 1.22102655e+04, 7.28174239e+03, 4.34255687e+03,
       2.58973734e+03, 1.54442180e+03, 9.21034985e+02, 5.49270571e+02,
       3.27564278e+02, 1.95346997e+02, 1.16497591e+02, 6.94747747e+01,
       4.14321386e+01, 2.47085668e+01, 1.47352585e+01, 8.78755312e+00,
       5.24056567e+00, 3.12527596e+00, 1.86379686e+00, 1.11149824e+00,
       6.62855683e-01, 3.95302161e-01, 2.35743318e-01, 1.40588435e-01,
       8.38416468e-02, 5.00000000e-02])

- RidgeCV ile cross-validation yöntemi ile bu lambda değerlerini modelimiz optimize ederek model için en iyi değeri bize verecek.

In [93]:
from sklearn.linear_model import RidgeCV

ridge_cv = RidgeCV(alphas = lamb_values,scoring='neg_mean_squared_error',normalize=True)

- RidgeCV modelimizi fit edip, oluşturuyoruz.

In [94]:
ridge_cv.fit(x_train,y_train)

RidgeCV(alphas=array([5.00000000e+09, 2.98181166e+09, 1.77824015e+09, 1.06047544e+09,
       6.32427608e+08, 3.77156003e+08, 2.24921633e+08, 1.34134790e+08,
       7.99929360e+07, 4.77047738e+07, 2.84493301e+07, 1.69661089e+07,
       1.01179482e+07, 6.03396320e+06, 3.59842837e+06, 2.14596713e+06,
       1.27977396e+06, 7.63208984e+05, 4.55149089e+05, 2.71433772e+05,
       1.61872877e+05, 9.65348864e+0...
       3.27564278e+02, 1.95346997e+02, 1.16497591e+02, 6.94747747e+01,
       4.14321386e+01, 2.47085668e+01, 1.47352585e+01, 8.78755312e+00,
       5.24056567e+00, 3.12527596e+00, 1.86379686e+00, 1.11149824e+00,
       6.62855683e-01, 3.95302161e-01, 2.35743318e-01, 1.40588435e-01,
       8.38416468e-02, 5.00000000e-02]),
        cv=None, fit_intercept=True, gcv_mode=None, normalize=True,
        scoring='neg_mean_squared_error', store_cv_values=False)

- Denenmesini istediğimiz lambda değerlerinden modelimiz için en iyi lambda değerini görelim 

In [95]:
ridge_cv.alpha_

0.05

## Model Tuned 

- Modelimiz için bulduğumuz en iyi lambda(**alpha**) parametre değeri ile yeniden Ridge Regression Modeli oluşturacağız

In [96]:
from sklearn.linear_model import Ridge

ridge_tuned = Ridge(alpha=0.05)

In [97]:
ridge_tuned.fit(x_train,y_train)

Ridge(alpha=0.05, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001)

- Aşağıda ki değerlere bakarak optimize ederek yeniden oluşturduğumuz Ridge Modelimiz biraz daha iyi

In [100]:
before_tuned = ridge_model.score(x_test,y_test)
after_tuned = ridge_tuned.score(x_test,y_test)

print(f"Ridge : {before_tuned} - Tuned Ridge : {after_tuned}")

Ridge : 0.9999999999720163 - Tuned Ridge : 0.9999999999930039


In [116]:
y_pred_tuned = ridge_tuned.predict(x_test)

- Mean Squared Error

In [117]:
from sklearn.metrics import mean_squared_error

mse_tuned = mean_squared_error(y_pred_tuned,y_test)

print(f"Mean Squared Error: {mse_tuned}")

Mean Squared Error: 1.9011703803370612e-09


In [118]:
np.sqrt(mse_tuned)

4.3602412551796504e-05